![sql](./img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. You need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use `JOIN` to merge tables along logical columns
    - Grouping Data with SQL, inlcuding `HAVING`
- Goal 5: Convert SQL to pandas

## Goal 1: Summarize

To date the data we've seen has looked like [this.](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm)

That is how we need data to look to run analysis and build models.<br>
But it doesn't _live_ there in it's native state.

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](./img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](./img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

## What does it mean to **Engineer Data**?

Let's start with a basic scenario:<br>
You are HR. You have *no* current database. 
How would you set it up?

Data needs considerations:
- What you want to store
- What "views" you anticipate wanting in the future

Structure considerations:
- Speed of retrieval
- How much data you are accessing
- How much you are storing

![etl](./img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](./img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
### SQL

[History of SQL](https://www.businessnewsdaily.com/5804-what-is-sql.html)

> Future users of large data banks must be protected from having to know how the data is organized in the machine (the internal representation). A prompting service which supplies such information is not a satisfactory solution. Activities of users at terminals and most application programs should remain unaffected when the internal representation of data is changed and even when some aspects of the external representation are changed. Changes in data representation will often be needed as a result of changes in query, update, and report traffic and natural growth in the types of stored information.
.
.
.<br>
<br>
In Section 1 a relational model of data is proposed as a basis for protecting users of formatted data systems from the potentially disruptive changes in data representation caused by growth in the data bank and changes in traffic.

[The original SQL paper from the 1970s.](https://www.seas.upenn.edu/~zives/03f/cis550/codd.pdf)

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](./img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

***
### SQLite

<img src="./img/SQLite-Python.jpg" width=500>

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to other database types there are many different libraries:
- Oracle: [cx_Oracle](https://oracle.github.io/python-cx_Oracle/)
- MySQL: [MySQL-python](https://github.com/farcepest/MySQLdb1)
- PostgreSQL: [Psycopg2](http://initd.org/psycopg/docs/)
- Microsoft SQL Server: [pymssql](http://www.pymssql.org/en/stable/)

These all implement [PEP 249: DB API v2](https://www.python.org/dev/peps/pep-0249/)

### The Structure of a SQL Query

<img src='./img/sql_statement.jpg'/>

***
## Goal 3: Get going with sqlite!

In [1]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

**Connection object** is your connection to the database, close that when you're done talking to the database all together. 

**Cursor object** is an iterator over a result set from a query. 

In [2]:
con = sqlite3.connect('data/flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

The `.excecute()` function will execute a sql statement and our `fetchall()` function will give us a list of all rows matching the execute statement.

In [3]:
# selecting the tables from the Master in the db
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


In [4]:
cursor.description

(('name', None, None, None, None, None, None),)

In [6]:
cursor.execute("SELECT name, type FROM sqlite_master;")
print(cursor.fetchall())

[('airports', 'table'), ('ix_airports_index', 'index'), ('airlines', 'table'), ('ix_airlines_index', 'index'), ('routes', 'table'), ('ix_routes_index', 'index')]


In [7]:
cursor.description

(('name', None, None, None, None, None, None),
 ('type', None, None, None, None, None, None))

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [8]:
cursor.execute("SELECT * FROM airports as a;").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [9]:
cursor.fetchone()

(1,
 '2',
 'Madang',
 'Madang',
 'Papua New Guinea',
 'MAG',
 'AYMD',
 '-5.207083',
 '145.7887',
 '20',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [10]:
cursor.fetchmany(4)

[(2,
  '3',
  'Mount Hagen',
  'Mount Hagen',
  'Papua New Guinea',
  'HGU',
  'AYMH',
  '-5.826789',
  '144.295861',
  '5388',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (3,
  '4',
  'Nadzab',
  'Nadzab',
  'Papua New Guinea',
  'LAE',
  'AYNZ',
  '-6.569828',
  '146.726242',
  '239',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (4,
  '5',
  'Port Moresby Jacksons Intl',
  'Port Moresby',
  'Papua New Guinea',
  'POM',
  'AYPY',
  '-9.443383',
  '147.22005',
  '146',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (5,
  '6',
  'Wewak Intl',
  'Wewak',
  'Papua New Guinea',
  'WWK',
  'AYWK',
  '-3.583828',
  '143.669186',
  '19',
  '10',
  'U',
  'Pacific/Port_Moresby')]

#### Use description

In [11]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [12]:
cursor.execute("PRAGMA table_info(airports);")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [13]:
cursor.execute("PRAGMA table_info(airlines);")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'alias', 'TEXT', 0, None, 0)
(4, 'iata', 'TEXT', 0, None, 0)
(5, 'icao', 'TEXT', 0, None, 0)
(6, 'callsign', 'TEXT', 0, None, 0)
(7, 'country', 'TEXT', 0, None, 0)
(8, 'active', 'TEXT', 0, None, 0)


In [14]:
cursor.execute("PRAGMA table_info(routes);")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'airline', 'TEXT', 0, None, 0)
(2, 'airline_id', 'TEXT', 0, None, 0)
(3, 'source', 'TEXT', 0, None, 0)
(4, 'source_id', 'TEXT', 0, None, 0)
(5, 'dest', 'TEXT', 0, None, 0)
(6, 'dest_id', 'TEXT', 0, None, 0)
(7, 'codeshare', 'TEXT', 0, None, 0)
(8, 'stops', 'TEXT', 0, None, 0)
(9, 'equipment', 'TEXT', 0, None, 0)


***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**Select**

**From**

**Where**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [18]:
cursor.execute("""
SELECT name
FROM airlines
WHERE active = 'Y'
  AND country = 'United Kingdom';
""")
cursor.fetchall()

[('Astraeus',),
 ('Air Southwest',),
 ('Aurigny Air Services',),
 ('Air Wales',),
 ('AD Aviation',),
 ('Air Foyle',),
 ('British Airways',),
 ('British International Helicopters',),
 ('bmi',),
 ('bmibaby',),
 ('British Midland Regional',),
 ('British Mediterranean Airways',),
 ('BA CityFlyer',),
 ('Crest Aviation',),
 ('Eastern Airways',),
 ('Excel Airways',),
 ('Excel Charter',),
 ('easyJet',),
 ('First Choice Airways',),
 ('Flightline',),
 ('Flybe',),
 ('Flyglobespan',),
 ('GB Airways',),
 ('Highland Airways',),
 ('Jet2.com',),
 ('Monarch Airlines',),
 ('MyTravel Airways',),
 ('Norfolk County Flight College',),
 ('Kinloss Flying Training Unit',),
 ('ScotAirways',),
 ('Thomas Cook Airlines',),
 ('Thomsonfly',),
 ('Virgin Atlantic Airways',),
 ('Jc royal.britannica',),
 ('Royal European Airlines',),
 ('CB Airways UK ( Interliging Flights )',),
 ('Air Cudlua',),
 ('BBN-Airways',),
 ('Rainbow Air Euro',),
 ('All Europe',)]

In [ ]:
cursor.execute(
"""
SELECT DISTINCT country
FROM airports;
"""
).fetchall()

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

<img src="./img/venn.png" width=550>

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [19]:
cursor.execute("""
SELECT *
FROM routes as r
JOIN airports as a
ON a.code=r.dest;
""").fetchmany(5)

cursor.description

(('index', None, None, None, None, None, None),
 ('airline', None, None, None, None, None, None),
 ('airline_id', None, None, None, None, None, None),
 ('source', None, None, None, None, None, None),
 ('source_id', None, None, None, None, None, None),
 ('dest', None, None, None, None, None, None),
 ('dest_id', None, None, None, None, None, None),
 ('codeshare', None, None, None, None, None, None),
 ('stops', None, None, None, None, None, None),
 ('equipment', None, None, None, None, None, None),
 ('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', No

### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

### Let's answer some questions

- Which countries have the highest amount of active airlines?
- Which countries have the highest amount of inactive airlines?

In [22]:
# Which countries have the highest amount of active airports?
cursor.execute("""
SELECT COUNT() as num, country
FROM airlines
WHERE active='Y'
GROUP BY country
HAVING num > 10
ORDER BY num DESC
LIMIT 30;
""")
print(*cursor.fetchall(), sep='\n')

(141, 'United States')
(72, 'Russia')
(40, 'United Kingdom')
(37, 'Germany')
(34, 'Canada')
(26, 'Australia')
(25, 'China')
(24, 'Spain')
(23, 'Brazil')
(22, 'France')
(19, 'Japan')
(18, 'Italy')
(17, 'Indonesia')
(17, 'India')
(16, 'Turkey')
(16, 'Thailand')
(15, 'Sweden')
(14, 'Switzerland')
(13, 'Ukraine')
(13, 'Portugal')
(12, 'Mexico')
(12, 'Finland')
(12, 'Egypt')
(12, 'Austria')
(11, 'Peru')


In [24]:
# Which countries have the highest amount of inactive airports?
cursor.execute("""
SELECT COUNT() num, country
FROM airlines
WHERE active='N'
GROUP BY country
HAVING num > 10
ORDER BY num DESC
LIMIT 30;
""")
print(*cursor.fetchall(), sep='\n')

(939, 'United States')
(427, 'Mexico')
(367, 'United Kingdom')
(284, 'Canada')
(158, 'Russia')
(142, 'Spain')
(97, 'France')
(94, 'Germany')
(81, 'South Africa')
(80, 'Nigeria')
(76, 'Ukraine')
(73, 'Kazakhstan')
(72, 'Italy')
(67, 'Australia')
(55, 'Sweden')
(47, 'Sudan')
(46, 'Switzerland')
(45, 'China')
(43, 'Netherlands')
(38, 'Austria')
(36, 'Egypt')
(35, 'Chile')
(35, 'Brazil')
(35, 'Angola')
(34, 'Colombia')
(32, 'Thailand')
(32, 'Portugal')
(32, 'Mauritania')
(31, 'Indonesia')
(29, 'Kyrgyzstan')


### Your turn!

In your group write some code to answer the following question:

- What are the number of airports for each time zone?  Each row should have a number of airports and a time zone.

In [31]:
# What about airports by timezones?
cursor.execute("""
SELECT timezone, COUNT() as num
FROM airports
GROUP BY timezone
ORDER BY num DESC
LIMIT 10;
""")
print(*cursor.fetchall(), sep='\n')

('America/New_York', 628)
('America/Chicago', 373)
('Europe/Berlin', 319)
('America/Anchorage', 258)
('Europe/Paris', 232)
('America/Los_Angeles', 226)
('Asia/Chongqing', 222)
('Europe/London', 193)
('America/Toronto', 159)
('Asia/Calcutta', 141)


In [32]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

### Using DB Browser for SQLite 

![](https://sqlitebrowser.org/images/screenshot.png)

DB Browser for SQLite is a high quality, visual, open source tool to create, design, and edit database files compatible with SQLite.

DB4S is for users and developers who want to create, search, and edit databases. DB4S uses a familiar spreadsheet-like interface, and complicated SQL commands do not have to be learned.

#### Let's practice a few queries in DB Browser for SQLite


## Goal 5: Transfering from sqlite to pandas

In [34]:
import pandas as pd
pd_con = sqlite3.connect("./data/flights.db")
df = pd.read_sql_query("select * from airports limit 5;", pd_con)
df

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

In [35]:
# Which countries have the highest amount of active airports?
sql = '''
SELECT COUNT() num, country
FROM airlines
WHERE active='Y'
GROUP BY country
HAVING num > 10
ORDER BY num DESC
LIMIT 30;
'''
pd.read_sql_query(sql, pd_con)

,num,country
0,141,United States
1,72,Russia
2,40,United Kingdom
3,37,Germany
4,34,Canada
5,26,Australia
6,25,China
7,24,Spain
8,23,Brazil
9,22,France


In [36]:
pd_con.close()

## Your Turn!

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

In your group, answer the following questions about the data:
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [71]:
conn = sqlite3.connect('data/Chinook_Sqlite.sqlite')
cur = conn.cursor()
cur.execute("SELECT * FROM Track").fetchall()
cur.description

(('TrackId', None, None, None, None, None, None),
 ('Name', None, None, None, None, None, None),
 ('AlbumId', None, None, None, None, None, None),
 ('MediaTypeId', None, None, None, None, None, None),
 ('GenreId', None, None, None, None, None, None),
 ('Composer', None, None, None, None, None, None),
 ('Milliseconds', None, None, None, None, None, None),
 ('Bytes', None, None, None, None, None, None),
 ('UnitPrice', None, None, None, None, None, None))

In [66]:
cur.execute("SELECT * FROM Album").fetchall()
cur.description

(('AlbumId', None, None, None, None, None, None),
 ('Title', None, None, None, None, None, None),
 ('ArtistId', None, None, None, None, None, None))

In [65]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())


[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]


In [49]:
# What are the max and min dates in the Invoice table?
sql = '''
SELECT max(InvoiceDate), min(InvoiceDate)
FROM Invoice
'''
pd.read_sql_query(sql, conn)

,max(InvoiceDate),min(InvoiceDate)
0,2013-12-22 00:00:00,2009-01-01 00:00:00


In [70]:
# What tables would you need to answer "what is your most popular track?"
sql = '''
SELECT COUNT() as m, Track.Name, Artist.Name
FROM Track as a
JOIN Album as b ON a.AlbumId = b.AlbumId
JOIN Artist as c ON b.ArtistId = c.ArtistId 
GROUP BY Name
ORDER BY m DESC;
'''
pd.read_sql_query(sql, conn)

DatabaseError: Execution failed on sql '
SELECT COUNT() as m, Track.Name, Artist.Name
FROM Track as a
JOIN Album as b ON a.AlbumId = b.AlbumId
JOIN Artist as c ON b.ArtistId = c.ArtistId 
GROUP BY Name
ORDER BY m DESC;
': no such column: Track.Name

In [ ]:
# What values would you need from each table?

In [ ]:
# Put it all together:
# You need to create a query that can rank tracks in term of popularity.

sql = '''
-- YOUR CODE HERE
'''
pd.read_sql_query(sql, conn)

In [ ]:
# Advanced: get the artist who sang the song!
sql = '''
-- YOUR CODE HERE
'''
pd.read_sql_query(sql, conn)